# Indice espectrales NDWI en GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [8]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccionar Imagenes Landsat RS

In [9]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
                  .select(["B2","B3","B4","B5","B6","B7"],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001)\
                  .reproject(crs = "EPSG:32719", scale = 30)

In [10]:
# Seleccionar una imagen Sentinel2
img_S2 = ee.Image("COPERNICUS/S2_SR/20190808T145729_20190808T145732_T19LCF")\
                  .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
                          ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
                  .multiply(0.0001) # 0 - 1

## Consultar nombre bandas y proyección

In [11]:
# Nombre de las bandas
print("Nombre bandas L8: ",img_L8.bandNames().getInfo())
print("Nombre bandas S2: ",img_S2.bandNames().getInfo())

Nombre bandas L8:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
Nombre bandas S2:  ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [12]:
# Consultar Proyeccion
print("Proyeccion L8: ",img_L8.projection().getInfo())
print("Proyeccion S2: ",img_S2.select(['BLUE']).projection().getInfo())

Proyeccion L8:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion S2:  {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 300000, 0, -10, 8600020]}


In [13]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

## Recortar imagen

In [14]:
# Recortar informacion con la zona de estudio
img_L8_clip = img_L8.clip(zona_ee)
img_S2_clip = img_S2.clip(zona_ee)

In [15]:
# Visualizar 
Map.addLayer(img_L8_clip, viz_es, name = "Landsat8 RS")
Map.addLayer(img_S2_clip, viz_es, name = "Sentinel2 RS")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Determinar NDWI

In [16]:
# Funcion para NDWI GAO - Muñoz Aguayo, 2015
def Calc_NDWI_gao(image):
    ndwi = image.expression("float(nir - swir1)/(nir + swir1)", {
        'nir': image.select('NIR'),
        'swir1': image.select('SWIR1')
    })
    return ndwi.rename("NDWI")

In [17]:
# Funcion para NDWI MF - McFeeters, 1996
def Calc_NDWI_mf(image):
    ndwi = image.expression("float(green - nir)/(green + nir)", {
        'green': image.select('GREEN'),
        'nir': image.select('NIR')
    })
    return ndwi.rename("NDWI")

In [18]:
# Calcular indices con funcion
ndwi_L8 = Calc_NDWI_mf(img_L8_clip)
ndwi_S2 = Calc_NDWI_mf(img_S2_clip)

In [19]:
# Simbologia de NDWI
viz_ndwi = {
    'palette': ['fff947', 'b9fa3e', '5ce5d3', '5172f4', '001334'],
    'min': -0.5,
    'max': 0.8,
    'bands': "NDWI"
}

In [20]:
# Visualizar 
Map.addLayer(ndwi_L8, viz_ndwi, name = "Landsat8 NDWI")
Map.addLayer(ndwi_S2, viz_ndwi, name = "Sentinel2 NDWI")
Map

Map(bottom=2248967.0, center=[-12.89439354983945, -69.99999046325685], controls=(WidgetControl(options=['posit…

## Clasificación NDWI

In [25]:
# Seleccionar
#  NDWI >= 0.1 es agua
ndwi_L8_Agua = ee.Image(0).where(ndwi_L8.lt(0.1),0)\
                          .where(ndwi_L8.gte(0.1),1)\
                          .clip(zona_ee)\
                          .reproject(crs="EPSG:32719", scale = 30)\
                          .rename("Agua")

ndwi_S2_Agua = ee.Image(0).where(ndwi_S2.lt(0.1),0)\
                          .where(ndwi_S2.gte(0.1),1)\
                          .clip(zona_ee)\
                          .reproject(crs="EPSG:32719", scale = 10)\
                          .rename("Agua")

In [26]:
ndwi_L8_Agua.getInfo()

{'type': 'Image',
 'bands': [{'id': 'Agua',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'crs': 'EPSG:32719',
   'crs_transform': [30, 0, 0, 0, -30, 0]}]}

In [27]:
ndwi_L8_Agua.bandNames().getInfo()

['Agua']

In [28]:
print(ndwi_L8_Agua.projection().getInfo())
print(ndwi_S2_Agua.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
{'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [10, 0, 0, 0, -10, 0]}


In [29]:
# Simbologia de clasificacion EVI
viz_agua = {
    'palette': ["#fdfdfd","blue"],
    'min': 0,
    'max': 1
}

In [30]:
# Visualizar 
Map.addLayer(ndwi_L8_Agua, viz_agua, name = "L8_Agua")
Map.addLayer(ndwi_S2_Agua, viz_agua, name = "S2_Agua")
Map

Map(bottom=562368.3635864258, center=[-12.86134318154514, -69.91218566894533], controls=(WidgetControl(options…

## Exportar Imagen

In [31]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [32]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [33]:
# Exportar imagen directorio Landsat
geemap.ee_export_image(ndwi_L8_Agua, filename="ndwi_L8_agua.tif", scale=30, region=geometria, file_per_band=False)
geemap.ee_export_image(ndwi_L8, filename="L8_ndwi.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\ndwi_L8_agua.tif
Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_ndwi.tif


In [34]:
# Exportar imagen directorio Sentinel-2
geemap.ee_export_image(ndwi_S2_Agua, filename="ndwi_S2_Agua.tif", scale=10, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\ndwi_S2_Agua.tif
Generating URL ...
An error occurred while downloading.
Total request size (98259616 bytes) must be less than or equal to 33554432 bytes.


In [35]:
# Exportarlo en google drivers
geemap.ee_export_image_to_drive(ndwi_S2, description='S2_ndwi', folder='GEE_Python', region=geometria, scale=10)

Exporting S2_ndwi ...
